In [4]:
import torch.nn as nn


/tmp/ipykernel_266327/287943212.py:23: UserWarning: Could not import flash_attn
  warnings.warn(colored("Could not import flash_attn", "magenta"))


In [8]:
# sequence = df['sequence'][:100]

In [9]:
# IGNORE_INDEX = -100
# PAD_INDEX = 0
# input_samples = []
# for seq in sequence:
#     length = len(seq)
#     seq_vocab = ['A', 'C', 'G', 'U', 'N']
#     seq_stoi = dict(zip(seq_vocab, range(len(seq_vocab))))
#     int_sequence = list(map(seq_stoi.get, seq))
#     input_sample = torch.LongTensor(int_sequence)
#     input_sample = {'src_seq': input_sample, 'length': torch.LongTensor([len(input_sample)])[0]}
#     input_samples.append(input_sample)
# collator = CollatorRNA(PAD_INDEX, IGNORE_INDEX)
# batch = collator(input_samples)

In [10]:
# pair_mask = make_pair_mask(batch['src_seq'], batch['length'])
# pair_mask.shape

In [11]:
# pair_latent = EmbedSequence2Matrix(CONFIG)(batch['src_seq'])
# pair_latent.masked_fill_(pair_mask[:, :, :, None], 0.0)
# pair_latent.shape

In [12]:
from pathlib import Path
import sys
sys.path.append('..')
import os
import pandas as pd
from rnacomp.dataset import LenMatchBatchSampler, RNA_DatasetBaselineSplitbppV0, DeviceDataLoader, RNA_DatasetBaselineSplitssV1

class CFG:
    path = Path("../data/")
    pathbb = Path("../data/Ribonanza_bpp_files")
    split_id = Path('../eda/fold_split.csv')
    pathss = Path("../eda/train_ss_vienna_rna.parquet")
    bs = 4
    num_workers = 8
    device = 'cpu'
    adjnact_prob = 0.5



fns = list(CFG.pathbb.rglob("*.txt"))
bpp_df = pd.DataFrame({"bpp": fns})
bpp_df['sequence_id'] = bpp_df['bpp'].apply(lambda x: x.stem)
ss = pd.read_parquet(CFG.pathss)[["sequence_id", "ss_full"]]
df = pd.read_parquet(CFG.path/'train_data.parquet')
split = pd.read_csv(CFG.split_id)
df = pd.merge(df, split, on='sequence_id')
df = pd.merge(df, bpp_df, on='sequence_id')
df = pd.merge(df, ss, on='sequence_id')
df_train = df.query('is_train==True').reset_index(drop=True)
df_valid = df.query('is_train==False').reset_index(drop=True)

ds_val = RNA_DatasetBaselineSplitssV1(df_valid, mode='eval')
ds_val_len = RNA_DatasetBaselineSplitssV1(df_valid, mode='eval', mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=CFG.bs, 
               drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, 
               batch_sampler=len_sampler_val, num_workers=CFG.num_workers), CFG.device)

batch = next(iter(dl_val))[0]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [13]:
with torch.no_grad():
    md = CustomRnaFormer(CONFIG)
    load_matching_weights(md, '/opt/slh/rna/eda/rna_former.pth')
    out = md(batch)


Matched weights: 118
Unmatched weights: 1


/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


In [14]:
out.shape

torch.Size([4, 206, 2])

In [89]:
mask.shape

torch.Size([4, 175, 175])

tensor([[1.5906, 4.1488, 3.2550,  ..., 2.4507, 0.6470, 2.0523],
        [1.6022, 3.6314, 2.8822,  ..., 2.2289, 0.9459, 2.8971],
        [1.9657, 4.1061, 2.8121,  ..., 2.5689, 1.0332, 2.0962],
        ...,
        [  -inf,   -inf,   -inf,  ...,   -inf,   -inf,   -inf],
        [  -inf,   -inf,   -inf,  ...,   -inf,   -inf,   -inf],
        [  -inf,   -inf,   -inf,  ...,   -inf,   -inf,   -inf]])

tensor([[ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        [ True,  True,  True,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])